## Кредитный скоринг. Первичный анализ <a id='start'></a>

**Задача:** Проанализировать предоставленные кредным отделом банка данные с целью выявления факторов, влияющих на надежность заемщика.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

#### Данное исследование разделим на несколько частей.

#### Часть 1. Изучение общей информации

 - [1. Изучение файлов с данными, получение общей информации, загрузка библиотек](#open_and_read)

#### Часть 2. Подготовка данных

- [1. Ликвидация пропусков и обработка аномалий](#missings)

- [2. Приведение данных к нужным типам](#another_type)

- [3. Поиск и ликвидация дубликатов](#dublicates)

- [4. Лемматизация данных](#lemmatize)

- [5. Категоризация данных](#category)

#### Часть 3. Выводы по результатам исследования

- [1. Ответы на поставленные в исследовании вопросы](#the_questions)

## Шаг 1. Знакомство с данными. Импорт библиотек. <a id='open_and_read'></a>

In [1]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')

df.info()

display(df.head(10))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


### Вывод

По результатам изучения входных данных имеем следующее:

- столбец 'children' после проверки обнаружился ряд протитиворечивых значений. 20 детей наблюдается у 76 заемщиков(проверить связь с другими данными). -1 ребенок у 47 человек (скорее всего это банальная опечатка, проверить)

- столбец 'days_employed' есть пропущенные значения. Есть данные со знаком + и - (разобраться почему). Есть ложные значения (строка 4). Почему тип float64(?), когда 'количество дней' - целое число. Привести к int64. Понять есть ли закономерность в пропусках

- столбец 'dob_years' 101 человек указал в качестве возраста 0 лет. Некорректно. Проверить, исправить

- столбец 'education' привести к единому регистру. Проверить нет ли орфографических ошибок в данных

- столбец 'education_id' на первый взгляд выглядит корректно. Возможных категорий 4. Уникальных значений 5. Проверить 4, 0 категории

- столбец 'family_status' привести к одному регистру, в остальном выглядит корректно. Видимо, при заполнении, пользователи выбирали уже готовый вариант из выпадающего списка(например)

- столбец 'family_status_id' на первый взгляд выглядит корректно

- столбец 'gender' стоило бы категоризировать. Так память расходовалась бы эффективнее. 1 заничение XNA - исправить

- столбец 'income_type' выглядит корректно (всего 8 категорий)
    
- столбец 'debt' на первый взгляд выглядит корректно, (возможно) привести к типу bool

- столбец 'total_income' есть пропущенные значения. На глазах - связь со столбцом 'days_employed'. ВЫЯСНИТЬ причину. ЗАПОЛНИТЬ

- столбец 'purpose' выглядит корректно, (возможно) категоризировать. Все цели кредитования можно разделить на 4 категории: недвижимость, автомобиль, свадьба, образование. Поэтому провести ЛЕММАТИЗАЦИЮ оставив всего 4 категории. Целям нашего проекта это не помешает

## Шаг 2. Предобработка данных

### Обработка пропусков <a id='missings'></a>

На основании связи days_employed и income_type можно сделать предварительный вывод о том, что люди намеренно скрывают свои доходы(предположение - безработные)

Посмотрим какую долю составляют эти данные и можно ли ими пренебречь:

In [2]:
df['days_employed'].isna().sum() / len(df) * 100 
#df['days_employed'].median()

10.099883855981417

Получаем 10% - достаточно много чтобы просто пренебречь данными. Нужно разбираться.

Посмотрим на другие данные в строках с пропущенными значениями. Возможно, удастся выявить некую закономерность пропусков

In [3]:
#df['days_employed'] = df['days_employed'].fillna('пусто')
#df[df['days_employed'] == 'пусто']

Никакой связи с другими данными обнаружено не было. Предварительный вывод - пропуски являются абсолютно случайными. Возможное решение - заполнить данные медианными значениями и продолжить исследование

In [4]:
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].median())

Проверим, изменилось ли количество не пустых значений в days_employed:

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Сразу проверим количество экстримальных значений в days_employed:

In [6]:
df.sort_values(by='days_employed').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
16335,1,-18388.949901,61,среднее,1,женат / замужем,0,F,сотрудник,0,186178.934089,операции с недвижимостью
4299,0,-17615.563266,61,среднее,1,женат / замужем,0,F,компаньон,0,122560.741753,покупка жилья
7329,0,-16593.472817,60,высшее,0,женат / замужем,0,F,сотрудник,0,124697.846781,заняться высшим образованием
17838,0,-16264.699501,59,среднее,1,женат / замужем,0,F,сотрудник,0,51238.967133,на покупку автомобиля
16825,0,-16119.687737,64,среднее,1,женат / замужем,0,F,сотрудник,0,91527.685995,покупка жилой недвижимости


In [7]:
18388 / 365

50.37808219178082

50 лет - максимальное значение в голове таблицы. Здесь все хорошо. Проверим хвост

In [8]:
df.sort_values(by='days_employed').tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7794,0,401663.850046,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,401674.466633,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,401675.093434,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,401715.811749,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6954,0,401755.400475,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


In [9]:
401663.850046 / 365

1100.4489042356163

Хм... 1100 лет - слишком много для одной человеческой жизни. И не слишком похоже, чтобы человек ввел такое значение намеренно. Возможно, ошибка кроется в неверном подсчете количества отработанных дней в самой программе. Проверим количество таких значений выходящих за пределы адекватности.

In [10]:
df[df['days_employed'] > (365*60)].count()

children            3445
days_employed       3445
dob_years           3445
education           3445
education_id        3445
family_status       3445
family_status_id    3445
gender              3445
income_type         3445
debt                3445
total_income        3445
purpose             3445
dtype: int64

In [11]:
3445 / 21525 * 100

16.004645760743323

Ух ты... Получается, что 16% заемщиков имеют более 60 лет стажа! Жаль, но маловероятно. Проверим связь с другими данными

In [12]:
df[df['days_employed'] > (365*60)].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
18,0,400281.136913,53,среднее,1,вдовец / вдова,2,F,пенсионер,0,56823.777243,на покупку подержанного автомобиля
24,1,338551.952911,57,среднее,1,Не женат / не замужем,4,F,пенсионер,0,290547.235997,операции с коммерческой недвижимостью
25,0,363548.489348,67,среднее,1,женат / замужем,0,M,пенсионер,0,55112.757732,покупка недвижимости
30,1,335581.668515,62,среднее,1,женат / замужем,0,F,пенсионер,0,171456.067993,операции с коммерческой недвижимостью
35,0,394021.072184,68,среднее,1,гражданский брак,1,M,пенсионер,0,77805.677436,на проведение свадьбы
50,0,353731.432338,63,среднее,1,женат / замужем,0,F,пенсионер,0,92342.730612,автомобили
56,0,370145.087237,64,среднее,1,вдовец / вдова,2,F,пенсионер,0,149141.043533,образование
71,0,338113.529892,62,среднее,1,женат / замужем,0,F,пенсионер,0,43929.696397,автомобили
78,0,359722.945074,61,высшее,0,женат / замужем,0,M,пенсионер,0,175127.646000,сделка с автомобилем


Наблюдается четкая связь со столбцом income_type. Все наблюдаемые мной заемщики принадлежат к категории "пенсионер". Посмотрим выборку конкретно по этой категории. 

In [13]:
df[df['income_type'] == "пенсионер"].describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,3856.000000,3856.000000,3856.000000,3856.000000,3856.000000,3856.000000,3443.000000
mean,0.132002,325780.609113,59.063019,0.913900,0.985737,0.056017,137127.465690
std,1.014271,114999.781015,7.618526,0.510185,1.315202,0.229984,80246.953231
min,-1.000000,-1203.369529,0.000000,0.000000,0.000000,0.000000,20667.263793
25%,0.000000,340199.573133,56.000000,1.000000,0.000000,0.000000,82881.443465
50%,0.000000,360505.668544,60.000000,1.000000,0.000000,0.000000,118514.486412
75%,0.000000,380751.301140,64.000000,1.000000,2.000000,0.000000,169700.433010
max,20.000000,401755.400475,74.000000,4.000000,4.000000,1.000000,735103.270167


Вывод 1: Уже на данном этапе видно, что медиана для всей таблицы не корректна в отношениии граждан из категории "пенсионер". Правильнее было бы найти свою медиану для каждой категории заемщиков из столбца income_type и заменить пустые значения

Вывод 2: Большая часть положительных значений в days_employed выглядит неадекватно. Но, данные одного порядка. Значит, есть логика. Ищем

In [14]:
df[(df['income_type'] == "пенсионер") & (df['days_employed'] > 0)].min()

children                               -1
days_employed                      328729
dob_years                               0
education                          ВЫСШЕЕ
education_id                            0
family_status       Не женат / не замужем
family_status_id                        0
gender                                  F
income_type                     пенсионер
debt                                    0
total_income                      20667.3
purpose                        автомобили
dtype: object

Минимальное положительное значение для категории "пенсионер" - 328729. Посмотрим как обстоят дела в других категориях

In [15]:
df[(df['income_type'] != "пенсионер") & (df['days_employed'] > 0)].min()

children                                  0
days_employed                        337524
dob_years                                31
education                            Высшее
education_id                              0
family_status              гражданский брак
family_status_id                          0
gender                                    F
income_type                     безработный
debt                                      0
total_income                          59957
purpose             покупка жилья для сдачи
dtype: object

Ок. Окончательно выяснено, что со всеми положительными значениями какая-то беда. Исправим ее

Предположение - необоснованно большие данные являюся количеством отработанных часов, а не дней. Проверим

In [16]:
df[df['days_employed'] > 0]['days_employed'].max() / 24 / 365

45.8624886387363

45 лет - уже более похоже на правду! Сравнивая с максимальным значением на противоположном конце списка (50 лет), ожидаешь увидеть что-то чуть более близкое, например 49 лет. Но это уже не первоначальные 1100 лет, так что, оставляем все как есть.

Посмотрим на минимальное положительное значение:

In [17]:
df[df['days_employed'] > 0]['days_employed'].min() / 24 / 365

37.526109658050025

Странно... Ожидал увидеть что-то близкое к 1 году. 37 лет - это немножко больше ожидаемого

А сколько вообще этих завышенных значений относятся к категории "пенсионер"?

In [18]:
df[df['days_employed'] > 0]['days_employed'].count()

3445

In [19]:
df[(df['income_type'] == "пенсионер") & (df['days_employed'] > 0)]['days_employed'].count()

3443

Вау, получается, что 99% странных завышенных данных относятся к категории "пенсионер". Значит, в полученных выше цифрах 37 и 45 лет трудового стажа нет ничего странного!

В таком случае, преобразуем "стаж в часах" в "стаж в днях"

In [20]:
df.loc[df['days_employed'] > 0, 'days_employed'] = df.loc[df['days_employed'] > 0, 'days_employed'] / 24

Проверим, как все получилось:

In [21]:
df.sort_values(by='days_employed').tail()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
7794,0,16735.993752,61,среднее,1,гражданский брак,1,F,пенсионер,0,48286.441362,свадьба
2156,0,16736.436110,60,среднее,1,женат / замужем,0,M,пенсионер,0,325395.724541,автомобили
7664,1,16736.462226,61,среднее,1,женат / замужем,0,F,пенсионер,0,126214.519212,операции с жильем
10006,0,16738.158823,69,высшее,0,Не женат / не замужем,4,F,пенсионер,0,57390.256908,получение образования
6954,0,16739.808353,56,среднее,1,вдовец / вдова,2,F,пенсионер,0,176278.441171,ремонт жилью


Ок, теперь столбец days_employed приведен в порядок. 

Можно переходить к заполнению пропусков в total_income:

Начнем с просмотра всех экстримальных значений

In [22]:
df.sort_values(by='total_income', ascending=False).head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12412,0,-1477.438114,44,высшее,0,женат / замужем,0,M,компаньон,0,2.265604e+06,ремонт жилью
19606,1,-2577.664662,39,высшее,0,женат / замужем,0,M,компаньон,1,2.200852e+06,строительство недвижимости
9169,1,-5248.554336,35,среднее,1,гражданский брак,1,M,сотрудник,0,1.726276e+06,дополнительное образование
20809,0,-4719.273476,61,среднее,1,Не женат / не замужем,4,F,сотрудник,0,1.715018e+06,покупка жилья для семьи
17178,0,-5734.127087,42,высшее,0,гражданский брак,1,M,компаньон,0,1.711309e+06,сыграть свадьбу


В максимальных значениях не наблюдается ничего странного. Посмотрим на минимумы

In [23]:
df.sort_values(by='total_income').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
14585,0,14967.460806,57,среднее,1,женат / замужем,0,F,пенсионер,1,20667.263793,недвижимость
13006,0,15404.524546,37,среднее,1,гражданский брак,1,M,пенсионер,0,21205.280566,заняться высшим образованием
16174,1,-3642.820023,52,Среднее,1,женат / замужем,0,M,сотрудник,0,21367.648356,приобретение автомобиля
1598,0,14988.587675,68,среднее,1,гражданский брак,1,M,пенсионер,0,21695.101789,на проведение свадьбы
14276,0,14441.768908,61,среднее,1,женат / замужем,0,F,пенсионер,0,21895.614355,недвижимость


Все выглядит нормально. Сравним значения mean и median 

In [24]:
df['total_income'].mean()

167422.30220817294

In [25]:
df['total_income'].median()

145017.93753253992

Разница не очень большая. Это говорит о том, что в списке нет сильно выбивающихся из общей массы значений. Для заполнения пропусков возьмем median() 

In [26]:
df['total_income'] = df['total_income'].fillna(df['total_income'].median())

Проверим количество пустых ячеек

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


С пропусками покончено!

### Вывод

Никакой логической связи пропусков с другими данными мной замечено не было. На мой взгляд, они являются абсолютно случайными.

Так же, в процессе работы над df обнаружился ряд экстримальных значений в days_employed напрямую связанных с income_type. Все завышенные значения принадлежали к одной категории - "пенсионер". Так как заемщики принадлежат к одной категории, смею предположить, что  либо, они неверно истрактовали требуемый запрос данных, либо данные ошибочно конвертируются на этапе обработки конкретно этой категории.

### Замена типа данных <a id='another_type'></a>

Начнем с того, какие данные требуют замены типа:

- столбец days_employed необходимо привести к int64. Потому, что количество дней - целое число. + Неплохо было бы заменить отрицательные значения - положительными

- столбец total_income тоже мог бы кушать меньше памяти если бы был int. В конечном итоге, влияние десятых долей рубля на результат будет насколько мало, что им можно пренебречь

- столбец debt. Здесь всего два возможных сценария: "Да" / "Нет". Поэтому имеет смысл перевести его в bool т. к. объем используемой памяти у bool в 8 раз меньше чем у int64.  

Приведем значения в days_employed к int64:

In [28]:
df['days_employed'] = df['days_employed'].astype('int')

In [29]:
df['days_employed'].dtypes

dtype('int64')

Готово. Теперь переведем отрицательные значения в положительные

In [30]:
df['days_employed'].head(3)

0   -8437
1   -4024
2   -5623
Name: days_employed, dtype: int64

In [31]:
df.loc[df['days_employed'] < 0, 'days_employed'] = df.loc[df['days_employed'] < 0, 'days_employed'] * (-1)

In [32]:
df['days_employed'].head(3)

0    8437
1    4024
2    5623
Name: days_employed, dtype: int64

Проверим остались ли отрицательные значения:

In [33]:
df.loc[df['days_employed'] < 0, 'days_employed']

Series([], Name: days_employed, dtype: int64)

Нет, теперь только положительные. Переходим к children и исправим отрицательные значения там:

In [34]:
df.loc[df['children'] < 0, 'children'] = df.loc[df['children'] < 0, 'children'] * (-1)

Проверим:

In [35]:
df['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

Отлично. Теперь Переходим к total_income и отбросим лишние нули:

In [36]:
df['total_income'] = df['total_income'].astype('int')

In [37]:
df['total_income'].head()

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

Здесь тоже избавились от лишних нулей, тем самым сделав таблицу читабельнее.

Остался debt. Пора перевести его в bool

In [38]:
df['debt'] = df['debt'].astype('bool')

In [39]:
df['debt'].dtypes

dtype('bool')

Отлично. Посмотрим на информацию преобразованного DF

In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null bool
total_income        21525 non-null int64
purpose             21525 non-null object
dtypes: bool(1), int64(6), object(5)
memory usage: 1.8+ MB


Объем использующейся памяти снизился на 10% благодаря смене типа данных столбца debt. Это ли не оптимизация)

Посмотрим на таблицу в целом. Как она изменилась?

In [41]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,False,145885,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,дополнительное образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,сыграть свадьбу


Таблица стала гораздо читабельнее. Теперь в данных прослеживается простая и понятная с первого взгляда логика

### Вывод

Здесь мы меняли типы данных преследуя 2 цели:

1. Избавиться от лишней нагроможденности цифрами, т.к. она мешает простому восприятию информации из таблицы. Увеличивает время ее обработки мозгом специалиста. При этом, не внося существенного вклада в конечный результат

2. Оптимизировать объем используемой памяти. Так DF будет кушать меньше мощностей, что увеличит скорость работы с таблицей. Нам удалось снизить потребляемый объем на 10%

### Обработка дубликатов <a id='dublicates'></a>

Начнем с того, что данные в столбце education приведем к одному регистру

In [42]:
df['education'] = df['education'].str.lower()

Проверим:

In [43]:
df['education'].head()

0     высшее
1    среднее
2    среднее
3    среднее
4    среднее
Name: education, dtype: object

Отлично. Теперь, посчитаем количество дубликатов в таблице встроенным методом .duplicated()

In [44]:
df.duplicated().sum()

71

Ок, уберем все дубликаты и переназначим индексы

In [45]:
df = df.drop_duplicates().reset_index(drop=True)

Проверим как все сработало:

In [46]:
df.duplicated().sum()

0

Мы избавились от всех дубликатов

### Вывод

Во время поиска дубликатов обнаружилось, что все они прячутся в тех строках, где ранее были пропущенные значения. Отсюда следует вывод, что часть из дубликатов, образовалась в процессе заполнения пустых ячеек средним. 

Другая часть - возможно, возникла из-за повторных отправок запросов на кредит со стороны заемщиков. 

Количество обнаруженных дубликатов составляет всего 0,3% от общей массы. Поэтому, простое пренебрежение этими данными не окажет значительного влияния на результат!

### Лемматизация <a id='lemmatize'></a>

В столбце purpose - мы видим огромное разнообразие целей получения кредита. Давайте взглянем на них:

In [47]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

Как видно из таблицы, 38 разнообразных целей, на самом деле крутятся вокруг всего 5-и категорий, это: образование, свадьба, авто, недвижимость. Приведем все цели к этим 4 категориям.

Для этого, сначала, необходимо вычленить леммы слов:

In [48]:
from pymystem3 import Mystem
m = Mystem()
for i in df['purpose']:
    lemmas = m.lemmatize(i)
    print(lemmas)

['покупка', ' ', 'жилье', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['покупка', ' ', 'жилье', '\n']
['дополнительный', ' ', 'образование', '\n']
['сыграть', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', '\n']
['операция', ' ', 'с', ' ', 'жилье', '\n']
['образование', '\n']
['на', ' ', 'проведение', ' ', 'свадьба', '\n']
['покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', '\n']
['покупка', ' ', 'недвижимость', '\n']
['покупка', ' ', 'коммерческий', ' ', 'недвижимость', '\n']
['сыграть', ' ', 'свадьба', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['покупка', ' ', 'жилой', ' ', 'недвижимость', '\n']
['строительство', ' ', 'собственный', ' ', 'недвижимость', '\n']
['недвижимость', '\n']
['строительство', ' ', 'недвижимость', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', '\n']
['недвижимость', '\n']
['приобретение', ' ', 'автомобиль', '\n']
['на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль',

Соответственно те строки, где есть упоминание "недвижимость" - отнесем к категории "недвижимость" и т.д.

### Вывод

Оказывается, все многообразие целей кредитования можно разделить всего на несколько простых категорий.

### Категоризация данных <a id='category'></a>

Категоризируем данные из столбца purpose. Напишем функцию и воспользуемся методом apply():

In [49]:
def to_category(obj):
    lemmas = m.lemmatize(obj)
    if 'жилье' in lemmas or 'недвижимость' in lemmas:
        return 'недвижимость'
    if 'авто' in lemmas or 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'образование' in lemmas:
        return 'образование'
    if 'свадьба' in lemmas:
        return 'свадьба'

In [50]:
df['purpose'] = df['purpose'].apply(to_category)

Проверим как изменились значения в слобце 'purpose'

In [51]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,False,253875,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,False,112080,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,False,145885,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,False,267628,образование
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,False,158616,свадьба


In [52]:
df.groupby('purpose').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income
purpose,,,,,,,,,,,
автомобиль,4306,4306,4306,4306,4306,4306,4306,4306,4306,4306,4306
недвижимость,10811,10811,10811,10811,10811,10811,10811,10811,10811,10811,10811
образование,4013,4013,4013,4013,4013,4013,4013,4013,4013,4013,4013
свадьба,2324,2324,2324,2324,2324,2324,2324,2324,2324,2324,2324


Категоризируем children. Наличие детей будет True а отсутствие False. На ее основе сделаем сводную таблицу.

In [53]:
def child_to_cat(obj):
    if obj > 0:
        return True
    if obj <= 0:
        return False

На основе таблицы df создадим новую, с данными которые нужны для ответа на поставленный вопрос

In [54]:
df_ch_debt = df[['children', 'debt']]

In [55]:
df_ch_debt.head()

,children,debt
0,1,False
1,1,False
2,0,False
3,3,False
4,0,False


Категоризируем наличие детей:

In [56]:
df_ch_debt['children'] = df_ch_debt['children'].apply(child_to_cat)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Проверим:

In [57]:
df_ch_debt.head()

,children,debt
0,True,False
1,True,False
2,False,False
3,True,False
4,False,False


Также, для ответа на поставленный в исследовании вопрос, необходимо категоризировать уровень дохода заемщиков:

In [58]:
df['total_income'].describe()

count    2.145400e+04
mean     1.652256e+05
std      9.802102e+04
min      2.066700e+04
25%      1.076230e+05
50%      1.450170e+05
75%      1.958132e+05
max      2.265604e+06
Name: total_income, dtype: float64

В качестве переходных границ из одной группы в другую будем использовать процентили 25% и 75% соответственно.

Тогда в группе со средним уровнем дохода окажется ровно 50% данных.

In [59]:
df.loc[((df['total_income'] >= 107623) & (df['total_income'] <= 195813)), 'total_income'].count()

10726

В группе с низким уровнем дохода окажется 25%

In [60]:
df.loc[(df['total_income'] < 107623), 'total_income'].count()

5364

В группе с высоким уровнем дохода тоже окажется 25%

In [61]:
df.loc[(df['total_income'] > 195813), 'total_income'].count() 

5364

Для ответа на поставленный вопрос нам этого будет достаточно.

Напишем функцию, которая будет присваивать ту или иную категорию в зависимости от уровня дохода:

In [62]:
def income_to_cat(obj):
    if obj < 107623:
        return 'низкий'
    if obj > 195813:
        return 'высокий'
    else:
        return 'средний'
    

Создадим новую таблицу

In [63]:
df_income = df[['debt', 'total_income']]

Присвоим категорию в отдельном столбце

In [64]:
df_income['income_status'] = df_income['total_income'].apply(income_to_cat)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


Проверим итоговую таблицу

In [65]:
df_income

,debt,total_income,income_status
0,False,253875,высокий
1,False,112080,средний
2,False,145885,средний
3,False,267628,высокий
4,False,158616,средний
...,...,...,...
21449,False,224791,высокий
21450,False,155999,средний
21451,True,89672,низкий
21452,True,244093,высокий


Сгруппируем данные по income_status

In [66]:
df_income.groupby('income_status').count()

,debt,total_income
income_status,,
высокий,5364,5364
низкий,5364,5364
средний,10726,10726


### Вывод

Обновив столбец целей, стало видно, что больше всего запросов на кредит с целью покупки недвижимости. Второе и третье место делят такие цели как: покупка автомобиля, и получение образования. Замыкает цепочку - свадьба. 

## Шаг 3. Ответьте на вопросы <a id='the_questions'></a>

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

Построим сводную таблицу, где и выясним связь между задолженностью и наличием детей

In [67]:
data_pivot_ch = df_ch_debt.pivot_table(index=['children'], values='debt', aggfunc='sum') / df_ch_debt.pivot_table(index=['children'], values='debt', aggfunc='count') * 100

In [68]:
data_pivot_ch

,debt
children,
False,7.543822
True,9.208203


### Вывод

Как видно из таблицы - вероятность своевременного возврата кредита у группы "без детей" выше на 1,7% чем у группы "с детьми". Разница не очень большая, но все же, можно утверждать, что зависимость имеет место быть. 

Эта разница может быть обусловлена тем, что наличие детей несет в себе бОльшуе финансовую нагрузку. Возможно, из-за этого не всегда остаются свободные деньги на погашение кредита, и появляются долги. 

### Есть ли зависимость между семейным положением и возвратом кредита в срок?

Создадим таблицу с интересующими нас данными

In [69]:
family_status_log = df[['family_status_id', 'debt']]

Создадим словарь пар "семейное положение" - "тип семейного положения"

In [70]:
family_status_dict = df[['family_status', 'family_status_id']]

In [71]:
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True).sort_values(by='family_status_id')

Посмотрим на получившийся словарь

In [72]:
family_status_dict

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Теперь создадим сводную таблицу как в предыдущем пункте и ответим на поставленный вопрос

In [73]:
pivot_family_status = family_status_log.pivot_table(index='family_status_id', values='debt', aggfunc='sum')

In [74]:
pivot_family_status

,debt
family_status_id,
0,931.0
1,388.0
2,63.0
3,85.0
4,274.0


Найдем доли людей из разных категорий имеющих задолженность

In [75]:
final_pivot_family_status = family_status_log.pivot_table(index='family_status_id', values='debt', aggfunc='sum') / family_status_log.pivot_table(index='family_status_id', values='debt', aggfunc='count') * 100

In [76]:
final_pivot_family_status

,debt
family_status_id,
0,7.545182
1,9.347145
2,6.569343
3,7.112971
4,9.750890


### Вывод

Мы видим, что больше всего должников в категории "Не женат / не замужем". Совсем рядом и люди из категории "гражданский брак". Самое минимальное количество должников в категории "вдовец / вдова". 

Есть ли зависимость между семейным положением и возвратом кредита в срок? Да, есть. Но совсем небольшая и, скорее, косвенная. 

Дело в том, что доля молодых людей в категории 4 и 1 выше, чем в категориях 0, 2, 3. А это, в свою очередь, означает чуть большую легкомысленность в принятии решений, меньший уровень дохода, меньшую стабильность жизни в целом. Эти факты уже напрямую влияют на вероятность возврата кредита в срок.

### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Для ответа на поставленный вопрос - сведем все необходимые данные в таблицу и вычислим доли должников в каждой из категорий

In [77]:
df_income_pivot = df_income.pivot_table(index='income_status', values='debt', aggfunc='sum') / df_income.pivot_table(index='income_status', values='debt', aggfunc='count') * 100

In [78]:
df_income_pivot

,debt
income_status,
высокий,7.140194
низкий,7.960477
средний,8.679843


### Вывод

Разница в вероятности возврата кредита в срок в зависимости от уровня дохода, составила менее 1%. Полученные данные говорят о том, что существенной зависимости между этими параметрами не наблюдается. 

### Как разные цели кредита влияют на его возврат в срок?

Воспользуемся ранее сгруппированными по целям данными.

In [79]:
purpose_pivot = df.pivot_table(index='purpose', values='debt', aggfunc='sum') / df.pivot_table(index='purpose', values='debt', aggfunc='count') * 100

In [80]:
purpose_pivot

,debt
purpose,
автомобиль,9.359034
недвижимость,7.233373
образование,9.220035
свадьба,8.003442


### Вывод

Небольшая разница в итоговых результатах есть. На ее основании можно утверждать, что связь целей и задолженности присутствует. Наимененьшее количество должников среди покупателей недвижимости. Мой вывод - недвижимость часто покупается с целью обретения источника пассивного дохода. Покупка автомобиля же, напротив, влечет за собой дополнительные траты на обслуживание и ремонт, и влияет на увеличение дохода у сравнительно небольшой группы лиц. Образование тоже не дает моментального увеличения количества поступающих денег. Его влияние будет оказано значительно позже и в нашу выборку не попадет. 

Что меня лично удивило, так это доля должников среди тех, кто брал деньги на празднование свадьбы. Само событие подразумевает трату денег сугубо в развлекательных целях. Мое предположение - часто, на свадьбах принято дарить молодоженам деньги. Также, часто финансовые обязательства берут на себя родители. Отсюда вывод, что относительно невысокий процент задолженности в этой группе связан с "отбиванием" части денег непосредственно на свадьбе. И помощь старшего поколения в погашении кредита.

## Шаг 4. Общий вывод

В целом, серьезной зависимости каких-либо категорий с возвратом кредита в срок не выявлено. Да, есть небольшая разница по каждой из проанализированных групп. Возможно, для более точной оценки потенциального заемщика нужно пользоваться совокупностью различных данных вместе. А уже оттуда выводить итоговую вероятность. Оценка потенциального заемщика только по одному из параметров может привести к ошибочному результату.

Жизнь людей сложна и многообразна чтобы можно было дать однозначный ответ опираясь только на несколько цифр из личной биографии.

[Вернуться в начало](#start)